In [1]:
import os
import cv2
import pickle
import imageio
import numpy as np

def psnr(img1, img2):
    mse = np.mean((img1 - img2) ** 2.0)
    if mse == 0:
        return 100.
    PIXEL_MAX = 255.0
    return 20 * np.log10(PIXEL_MAX / np.sqrt(mse))

def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [36]:
# Replace 'input_image.png' and 'compressed_image.bpg' with your file paths
base_dir = 'E:/PHD_DOC/RES1_SemanticCommunication5/Code/Benchmark/test_bpg_Kodak'
aver_len = 0
aver_psnr = 0
test_exp_num = 24
for i in range(test_exp_num):
    # index = np.random.randint(1, 10000)
    ####  修改编码方式 可以采用不同的信源编码 JPEG  JPEG200   
    ####  编码方式修改为 cv2.imwrite('saved_image.jpg',image,[cv2.IMWRITE_JPEG_QUALITY,95])，类似下面 -q的值
    index = i + 1
    input_image_path = f'{base_dir}/Kodak/kodim{index:02d}.png'
    bit_stream_path = f'{base_dir}/Kodak_compress_bitstream/kodim{index:02d}.txt'
    # Encode the input image to BPG QP 0~50
    encode_command = f'{base_dir}/bpg_lib/bpgenc -e x265 -q 31 -f 444 -o {bit_stream_path}  {input_image_path}'
    os.system(encode_command)

    # Decode the BPG image back to PNG
    decoded_image_path = f'{base_dir}/Kodak_compress/kodim{index:02d}-compressed.png'
    decode_command = f'{base_dir}/bpg_lib/bpgdec -o {decoded_image_path} {bit_stream_path}'
    os.system(decode_command)

    # 打开一个文件（如果文件不存在，将创建一个新文件）
    file_path = bit_stream_path
    file = open(file_path, "rb")  # 使用 "r" 模式表示只读模式  读取文件的内容
    file_contents = file.read()
    # print(len(file_contents))
    aver_len = aver_len + len(file_contents)
    file.close()
    
    img1 = cv2.imread(input_image_path)
    img2 = cv2.imread(decoded_image_path)
    
    aver_psnr = aver_psnr + psnr(img1 + 0., img2 + 0.)
    
    if i % 4==0:
        print('Average length:{:.2f}'.format(aver_len/(i+1)))
        print('Average PSNR:{:.2f}'.format(aver_psnr/(i+1)))
print('Overall Average length:{}'.format(aver_len/test_exp_num))
print('Overall Average PSNR:{}'.format(aver_psnr/test_exp_num ))


Average length:63164.00
Average PSNR:34.17
Average length:38841.40
Average PSNR:35.36
Average length:38339.00
Average PSNR:35.56
Average length:39680.62
Average PSNR:35.51
Average length:38078.24
Average PSNR:35.54
Average length:38089.81
Average PSNR:35.53
Overall Average length:37967.083333333336
Overall Average PSNR:35.54508243585998


In [33]:

for ratio in [0.06, 0.02, 0.1]:
    print( 512*768*3  * ratio * 4. *2./3. /8.)

23592.960000000003
7864.32
39321.6


In [38]:
# Multi Hop Test
base_dir = 'E:/PHD_DOC/RES1_SemanticCommunication5/Code/Benchmark/test_bpg_Kodak'
aver_len = 0
aver_psnr = 0
test_exp_num = 1
for i in range(test_exp_num):
    # index = np.random.randint(1, 10000)
    ####  修改编码方式 可以采用不同的信源编码 JPEG  JPEG200
    index = i + 1
    input_image_path = f'{base_dir}/Kodak/kodim{index:02d}.png'
    bit_stream_path = f'{base_dir}/Kodak_compress_bitstream/kodim{index:02d}.txt'
    # Encode the input image to BPG QP 0~50
    encode_command = f'{base_dir}/bpg_lib/bpgenc -e x265 -q 31 -f 444 -o {bit_stream_path}  {input_image_path}'
    os.system(encode_command)
    for n in range(100):
        # Decode the BPG image back to PNG
        decoded_image_path = f'{base_dir}/Kodak_compress/kodim{index:02d}-compressed.png'
        decode_command = f'{base_dir}/bpg_lib/bpgdec -o {decoded_image_path} {bit_stream_path}'
        os.system(decode_command)

        # 打开一个文件（如果文件不存在，将创建一个新文件）
        file_path = bit_stream_path
        file = open(file_path, "rb")  # 使用 "r" 模式表示只读模式  读取文件的内容
        file_contents = file.read()
        print(len(file_contents))
        file.close()
        img1 = cv2.imread(input_image_path)
        img2 = cv2.imread(decoded_image_path)
        print(psnr(img1 + 0., img2 + 0.))
        encode_command = f'{base_dir}/bpg_lib/bpgenc -e x265 -q 31 -f 444 -o {bit_stream_path}  {decoded_image_path}'
        os.system(encode_command)
    if i % 4==0:
        print('Average length:{:.2f}'.format(aver_len/(i+1)))
        print('Average PSNR:{:.2f}'.format(aver_psnr/(i+1)))
print('Overall Average length:{}'.format(aver_len/test_exp_num))
print('Overall Average PSNR:{}'.format(aver_psnr/test_exp_num ))


63164
34.16734873040945
61779
33.541268831776854
61179
33.18977212616661
60893
32.99281934954092
60702
32.84650709885207
60474
32.68117683375628
60390
32.569954027296134
60304
32.490636452236735
60258
32.44328055431447
60231
32.430388339732225
60224
32.4173923267056
60216
32.41283095741772
60216
32.4091160137806
60212
32.40960784280256
60215
32.40626375446245
60213
32.40697767845222
60214
32.405830825843374
60211
32.40660183137373
60214
32.40514614974429
60211
32.40470170988802
60212
32.401110921555876
60209
32.400856131826366
60212
32.40039855164977
60209
32.400833891331054
60212
32.40046133076455
60209
32.40079334700436
60212
32.400439092290945
60209
32.400856131826366
60212
32.40039855164977
60209
32.400833891331054
60212
32.40046133076455
60209
32.40079334700436
60212
32.400439092290945
60209
32.400856131826366
60212
32.40039855164977
60209
32.400833891331054
60212
32.40046133076455
60209
32.40079334700436
60212
32.400439092290945
60209
32.400856131826366
60212
32.40039855164977
60

KeyboardInterrupt: 